In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "../scripts")
import util
import camera
from sklearn.cluster import DBSCAN

np.set_printoptions(precision=4)
!pwd

/home/zpyang/git/visnet/src/visnet/notebooks


In [2]:
class Track:
    """
    This is a data structure for representing tracked targets
    """
    def __init__(self, n_particles, bounds) -> None:
        self.n_particles = n_particles
        self.bounds = bounds
        self.map_state = None # MAP estimate, particle with the max weight
        self.mean = None
        self.cov = None
        self.weights = np.ones(n_particles)
        self.particles = None

        self.cam_group = None
        self.hypothesis = None
        self.map_hypo = None 

    def get_mean(self):
        pass

In [3]:
def target_traj_circle(t):
    x = 15*np.sin(2 * np.pi * 0.01 * t / 3)
    y = 10*np.cos(2 * np.pi * 0.01 * t / 3)
    z = 20
    return np.array([x,y,z])

def target_traj_straight(t, start, finish, tf):
    max_dist = np.linalg.norm(start-finish)
    v_max = (finish-start) / tf
    # x = -15 + t * 0.1
    # y = 0
    # z = 20
    return start + v_max * t

def target_stationary(start):
    return start

In [8]:
class ParticleFilter():
    """
    unknown number of targets
    """
    def __init__(self,
                 n_targets=1,
                 n_particles=500,
                 spawn_regions=None,
                 cam_group=None,
                 n_eff_thresh=1,
                 sigma=150,
                 resample_proportion=None,
                 verbose=False,
                ):
        self.particles = None
        self.clusters = None
        self.weights = np.ones(n_particles)
        self.n_particles = n_particles
        self.n_targets = n_targets
        self.spawn_regions = spawn_regions
        self.exit_region = None
        self.n_eff_thresh = n_eff_thresh
        self.cam_group = cam_group
        self.sigma = sigma
        self.resample_proportion = resample_proportion or 0.0
        self.verbose = verbose
        self.tracks = []
        self.init_filter()
        
    def init_filter(self, mask=None):
        new_sample = self.prior_fn(self.n_particles, spawn_regions=self.spawn_regions)
        if mask is None:
            self.particles = new_sample
        else:
            self.particles[mask, :] = new_sample[mask, :]


    def update(self, observation=None):
        self.particles = self.dynamics_d(self.particles)
        
        self.hypothesis = self.observe_fn().reshape(self.n_particles, self.cam_group.n_cams * 2)

        self.weights = self.weight_fn(observation, self.hypothesis)      

        self.n_eff = (1.0 / np.sum(self.weights ** 2)) / self.n_particles
        # self.weight_entropy = np.sum(self.weights * np.log(self.weights, where=self.weights!=0))|

    def resample(self, resample_fn):
        
        if self.n_eff < self.n_eff_thresh:
            indices = resample_fn(self.weights)
            self.particles = self.particles[indices, :]
            self.weights = np.ones(self.n_particles) / self.n_particles
        
        # randomly resample some particles from the prior
        if self.resample_proportion > 0:
            random_mask = (
                np.random.random(size=(self.n_particles,)) < self.resample_proportion
            )
            self.resampled_particles = random_mask
            self.init_filter(mask=random_mask)

    def observe_fn(self):
        """
        Parameters:
            particles : all particles at a step
        Returns:
            hypothesis from x using the camera projection matrix
        """

        hypothesis = []
        for p in self.particles:
            pos = p[0:3]
            h = self.cam_group.get_group_measurement(pos, [1], hypo_mode=True)
            h = np.array(h)
            hypothesis.append(h)
        hypothesis = np.array(hypothesis)
        return hypothesis


    def weight_fn(self, z_set, hypo):
        """
        Iterate through each camera, find the measurement most likely to be associated with the target
        @ z_set: [{yj}_0, {yj}_1, ...{yj}_n]    n cameras
        @ hypo: [bearing_0, bearing_1, ....bearing_n]   n_cameras
        """

        weights = np.ones(self.n_particles)
        for i, zi in enumerate(z_set): # loop through cameras
            if self.verbose:
                print("i, zi", (i,zi.shape))
            
            wi = np.ones(self.n_particles)/self.n_particles
            hi = hypo[:, i*2:i*2+2]
            for yj in zi: # loop through measurements on each camera
                
                w_tmp = util.squared_error(hi, yj, sigma=self.sigma)
                S_tmp = sum(w_tmp)
                if self.verbose:
                    print("w_tmp", w_tmp.shape)
                    print("S_tmp", S_tmp)
                
                wi += w_tmp
            
            wi = wi/np.sum(wi)
            weights *= wi
            
        if self.verbose:
                print("weight: ", weights.shape)
        return weights / np.sum(weights)


    def prior_fn(self, n, spawn_regions=None):
        if spawn_regions is not None:
            n_spawn = spawn_regions.shape[0]
            for region in spawn_regions:
                pos = np.random.normal(region, 2*np.ones(3), (n_particles/n_spawn, 3))
                vel = np.random.normal(np.zeros(3), np.ones(1), (n_particles/n_spawn, 3))
                # label = np.random.randint(0, 2, (n,1))
                particles = np.hstack([pos, vel])
            return particles
        else:
            pos = np.random.uniform(low=[-20, -20, 5], high=[20, 20, 25], size=(n,3))
            vel = np.random.uniform(low=np.zeros(3), high=np.ones(3), size=(n,3))

            # label = np.random.randint(0, 2, (n,1))
            particles = np.hstack([pos, vel])
            return particles

    def cluster_fn(self, eps=2, min_samples=30):
        p = self.particles[:,0:3]
        w = self.weights
        data = np.c_[p,w]

        db = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
        labels = db.labels_
        unique_labels = set(labels)
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True

        clusters = []
        for k in unique_labels:
            class_member_mask = labels == k
            indice = class_member_mask & core_samples_mask
            cluster = data[indice]
            hypo = self.hypothesis[indice]

            if cluster.shape[0] != 0:

                argmax_weight = np.argmax(cluster[:, -1])
                map_state = cluster[argmax_weight]
                map_hypo = hypo[argmax_weight]
            else:
                map_state = None
                map_hypo = None

            

            self.tracks.append((map_state, map_hypo))

            clusters.append(cluster)
        
        self.clusters = clusters


    dt = 1/10
    A = np.block([
        [np.eye(3), dt*np.eye(3)],
        [np.zeros((3,3)), np.eye(3)]
    ])
    # A = np.block([
    #     [A, np.zeros((6,1))],
    #     [np.zeros((1,6)), 1]
    # ])
    @classmethod
    def dynamics_d(self, x):
        """
        Discrete dynamics, last state is the target id
        """
        n, d = x.shape
        w = np.zeros((d, n))
        w[3:6] = np.random.normal(0, 1, (3,n))
        x_1 = ParticleFilter.A @ x.T + w

        return x_1.T

In [9]:
start = np.array([-20, 0, 15])
end = np.array([20, 0, 15])

n_targets = 2
pfs = []

particles = []
state = []
state_est = []
n_eff = []
weights = []
clusters_list = []

t_final = 100

verbose = False
spawn_region = start
group = camera.CamGroup(camera.cam_param, camera.cam_poses[0:4,:])
n_particles = 2000

pf1 = ParticleFilter(
    n_particles=n_particles,
    # spawn_region=start,
    cam_group=group,
    n_eff_thresh=.8,
    sigma=100,
    n_targets=n_targets,
    resample_proportion=0.2,
    verbose=verbose)

for t in range(t_final):
    target_1 = target_traj_straight(t, start, end, t_final)
    target_2 = target_traj_straight(t, end, start, t_final)
    # target_3 = 
    # target_2 = target_traj_circle(t)
    # target_1 = target_stationary(np.array([-5,5,20]))
    # target_2 = target_stationary(np.array([-5,-5,20]))
    targets = np.vstack([target_1, target_2])
    # targets = np.vstack([target_1])

    state.append(targets)
    
    #each camera can give multiple measurements
    z = group.get_group_measurement(targets, [1,1])

    pf1.update(z)
    pf1.cluster_fn(eps=3, min_samples=10)
    
    particles.append(pf1.particles)
    clusters_list.append(pf1.clusters)

    # state_est.append(pf1.map_state)
    weights.append(pf1.weights)
    # cov.append(pf1.cov_state)
    pf1.resample(util.residual_resample)
    # pf1.resample(util.residual_resample)
    
    n_eff.append(pf1.n_eff)

state = np.array(state)
print(state.shape)
state = state.reshape(t_final, n_targets*3)
n_eff = np.array(n_eff)

particles = np.array(particles)
# state_est = np.array(state_est)
weights = np.array(weights)
# err1 = state[:,0:3] - state_est[:, 0:3]

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)

In [12]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib import cm
# Attaching 3D axis to the figure
fig = plt.figure("Weight visualization and Clustering", figsize=(20, 10))

ax1 = fig.add_subplot(1, 3, 1, projection="3d")
ax2 = fig.add_subplot(1, 3, 2, projection="3d")
ax3 = fig.add_subplot(1, 3, 3, projection="3d")
ax1.view_init(30,0)

def update_frame(i):
    # NOTE: there is no .set_data() for 3 dim data...
    x = particles[i, :, 0]
    y = particles[i, :, 1]
    z = particles[i, :, 2]
    w = weights[i, :]
    
    ax1.cla()
    ax1.set_xlim3d(-25,25)
    ax1.set_ylim3d(-25,25)
    ax1.set_zlim3d(0,25)
    ax1.scatter(x, y, z, cmap=cm.turbo, c=w, s=2)
    ax1.set_xlabel('x, m')
    ax1.set_ylabel('y, m')
    ax1.set_zlabel('z, m')
    
    ax2.cla()
    ax2.set_xlim3d(-25,25)
    ax2.set_ylim3d(-25,25)
    ax2.set_zlim3d(0,25)
    ax2.set_xlabel('x, m')
    ax2.set_ylabel('y, m')
    ax2.set_zlabel('z, m')

    clusters_i = clusters_list[i]
    colors = [plt.cm.tab10(each) for each in np.linspace(0, 1, len(clusters_i))]
    for cluster, col in zip(clusters_i, colors):
        ax2.scatter(
            cluster[:, 0],
            cluster[:, 1],
            cluster[:, 2],
            "o",
            color=col,
        )
        
    ax3.cla()
    ax3.set_xlim3d(-25,25)
    ax3.set_ylim3d(-25,25)
    ax3.set_zlim3d(0,25)
    ax3.set_xlabel('x, m')
    ax3.set_ylabel('y, m')
    ax3.set_zlabel('z, m')
    # print(pf1.tracks[i][0])
    if pf1.tracks[i][0] is not None:
        
        ax3.scatter(pf1.tracks[i][0][0], pf1.tracks[i][0][1], pf1.tracks[i][0][2])
    return None


ani = FuncAnimation(
    fig, update_frame, t_final, fargs=[],
    interval=int(1000 / 10), blit=False)
# ani.save("../videos/particles.mp4")
# HTML(ani.to_html5_video())
# plt.close()

In [108]:
print(state.shape)
plt.figure(figsize=(10,10))
ax = plt.axes(projection="3d")
ax.plot3D(state[:,0], state[:,1], state[:,2], marker='+', label='truth1', markersize=10)
# ax.plot3D(state[:,3], state[:,4], state[:,5], '+-', label='truth2')
ax.scatter(state_est[:,0], state_est[:,1], state_est[:,2], marker='x', label='est')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_xlim3d(-25,25)
ax.set_ylim3d(-25,25)
ax.set_zlim3d(0,25)
ax.view_init(60,-90)

plt.legend()

plt.figure(figsize=(10,8))
plt.subplot(3,1,1)
plt.plot(err1)
plt.title('error plot')
plt.xlabel('# of frames')
plt.ylabel('error[m]')

plt.subplot(3,1,2)
plt.plot(n_eff)
plt.title('n_eff')
plt.xlabel('# of frames')
plt.ylabel('n_eff')

plt.subplot(3,1,3)
plt.plot(state_est[:,-1])
plt.title('label')
plt.xlabel('# of frames')
plt.ylabel('label')
plt.show()

(200, 6)
